In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
np.random.seed(1)

In [2]:
def add_blob(points, mean, cov, n, min_distance=0.3):
    count = 0
    while count < n:
        point = np.random.multivariate_normal(mean, cov)
        if len(points) > 0:
            distances = np.linalg.norm(np.array(points) - np.expand_dims(point, 0), axis=1)
            dist_min = distances.min()
            if dist_min < min_distance:
                continue
        points.append(point)
        count += 1

points = []
add_blob(points, [0, 0], [[0.1, 0], [0, 0.1]], n=20)
add_blob(points, [4, 0], [[1, -1], [0, 0.1]], n=40)
add_blob(points, [3, -1.5], [[0.1, 0], [0.5, 0.5]], n=40)
data = np.array(points)
mean = data.mean(axis=0)
data -= mean

/var/folders/j_/vg6ptnsn1799lx1pvbnxqksr0000gn/T/ipykernel_10475/1474646037.py:4: RuntimeWarning: covariance is not positive-semidefinite.
  point = np.random.multivariate_normal(mean, cov)


In [3]:
def plot_data_points(data, center, given_distance, positive_distance, showlegend=False):
    center = np.array(center) - mean
    distances = np.linalg.norm(data - np.expand_dims(center, axis=0), axis=1)

    given_mask = np.zeros(len(data))
    given_mask[np.argmin(distances)] = 1
    given_mask = np.array(given_mask, dtype=bool)
    positive_mask = (distances < positive_distance) & (~given_mask)
    negative_mask = ~(given_mask | positive_mask)

    def scatter(d, color, symbol, size, name):
        return go.Scatter(
            x=d[:, 0], y=d[:, 1], name=name,
            mode='markers', marker=dict(color=color, symbol=symbol, size=size),
            legendgroup='maingroup', showlegend=showlegend)

    return [
        scatter(data[given_mask], color='#FFB74D', symbol='circle', size=5, name='Anchor'),
        scatter(data[positive_mask], color='#66BB6A', symbol='diamond', size=5, name='Positives'),
        scatter(data[negative_mask], color='#EF5350', symbol='x', size=6, name='Negatives')
    ]

In [4]:
fig = make_subplots(rows=1, cols=2, shared_xaxes=True, shared_yaxes=True)
for trace in plot_data_points(data, center=[3.5, -1], given_distance=0.75, positive_distance=1.5, showlegend=True):
    fig.add_trace(trace, row=1, col=1)
for trace in plot_data_points(data, center=[1, 0], given_distance=1, positive_distance=1.5):
    fig.add_trace(trace, row=1, col=2)
axis_layout = dict(zeroline=True, zerolinecolor='#BDBDBD', zerolinewidth=1,
                   gridwidth=0, showticklabels=False)
fig.update_layout(
    autosize=False, width=550, height=200, margin=dict(t=0, l=0, b=0, r=0),
    xaxis=axis_layout, yaxis=axis_layout, xaxis2=axis_layout, yaxis2=axis_layout,
    legend=dict(font=dict(color='black')),
    plot_bgcolor='white',
)#paper_bgcolor='rgba(0,0,0,0)')
fig.write_image('constrastive_learning_intro.pdf')
fig